In [1]:
import os
import pandas as pd
from adding_an_addition_following_the_index import adding_an_addition
from functions import change_Muni_Heb_to_Muni_Eng
from division_into_traffic_zones_of_plans import division_into_traffic_zones
from export_geo_layer_for_client_control import export_geo_layer
from export_index_layer_for_client_control import export_index_layer
from forecast import clientTaz
from geographical_features import add_geographical_Features
from index_layer import index_layer_fun
from status_exists_for_control import export_status_exists
from uploading_index_table_elements import uploading_index_table
from uploading_index_table_elements import uploading_index_table
from export_forecast_2040 import export_forecast

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [4]:
cwd = os.getcwd()
software_data_folder_location = os.path.dirname(cwd)
software_data_folder_location
df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_data_folder_location))

In [5]:
create_forecast_basic_location=df_inputs_outputs['location'][0]
forecast_version_basic_folder_location=df_inputs_outputs['location'][4]
client_data_folder_location=df_inputs_outputs['location'][1]
forecast_version=df_inputs_outputs['location'][2]
v_date=df_inputs_outputs['location'][3]
index_file_name='index_format_for_creating_forecast_jtmt_input_{}_{}'.format(forecast_version,v_date)

In [6]:
from functions import up_load_gdb, make_point, find_files_with_pattern, up_load_shp

In [7]:
# העלת אזורי תנועה לחישוב
folder_path=r'{}\For_approval\Reference_tabels\shp'.format(client_data_folder_location)

gpd_name='tochnit_check.gdb'

forecast = up_load_gdb(r'{}\{}'.format(folder_path,gpd_name),'TAZ_211028_V3_Published_with_client_changes')

In [8]:
#### הוספת מאפיינים גיאוגרפים לאזורי תנועה
forecast_point = make_point(forecast)

    # Load data layers
DISTRICT = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_DISTRICT')
urban = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_urban')
SCHOOLDISTRICT = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_SCHOOLDISTRICT')
PUMA = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_PUMA')
jerusalem_city = up_load_gdb(
        r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_data_folder_location), 'TAZ_V3_2_220123_In_jerusal')
subdistrict_il = up_load_gdb(
        r'{}\background_files\subdistrict2008.gdb'.format(software_data_folder_location), 'subdistrict2008_ITM')
muni_JTMT = up_load_gdb(
        r'{}\background_files\MUNI_border.gdb'.format(software_data_folder_location), 'muni_under_JTMT_ITM')
    
# מחפש קבצים של jeru_metro_jtmt_border
jeru_metro_jtmt_border_files = find_files_with_pattern(r'{}\background_files'.format(software_data_folder_location), 'jeru_metro_jtmt_border')
filepath = ''

# לוקח את הקובץ של השכבה
for string in jeru_metro_jtmt_border_files:
    if string.endswith('.shp'):
        filepath = string

# מעלה את השכבה
jeru_metro_jtmt_border = up_load_shp(filepath)

# Geographical join between traffic zones and data layers
forecast_point_DISTRICT = forecast_point.sjoin(
        DISTRICT)[['Taz_num', 'puma2040_csv_DISTRICT']]
forecast_point_urban = forecast_point.sjoin(
        urban)[['Taz_num', 'BaseProjections2040_csv_urban']]
forecast_point_SCHOOLDISTRICT = forecast_point.sjoin(
        SCHOOLDISTRICT)[['Taz_num', 'puma2040_csv_SCHOOLDISTRICT']]
forecast_point_PUMA = forecast_point.sjoin(
        PUMA)[['Taz_num', 'puma2040_csv_PUMA']]
forecast_point_jerusalem_city = forecast_point.sjoin(
        jerusalem_city)[['Taz_num', 'jerusalem_city']]
forecast_point_subdistrict_il = forecast_point.sjoin(
        subdistrict_il[['geometry', 'ENG_NAME_nafa']])[['Taz_num', 'ENG_NAME_nafa']]
forecast_point_muni_JTMT = forecast_point.query('main_sector!="Palestinian"').sjoin(
        muni_JTMT[['Muni_Heb', 'Sug_Muni', 'CR_PNIM', 'geometry']])[['Taz_num', 'Muni_Heb', 'Sug_Muni', 'CR_PNIM']]
forecast_point_jeru_metro_jtmt_border = forecast_point.sjoin(
        jeru_metro_jtmt_border)[['Taz_num', 'jeru_metro']]

# Merge tables into one table
forecast = (forecast
                .merge(forecast_point_subdistrict_il, on='Taz_num', how='left')
                .merge(forecast_point_muni_JTMT, on='Taz_num', how='left')
                .merge(forecast_point_jeru_metro_jtmt_border, on='Taz_num', how='left')
                .merge(forecast_point_DISTRICT, on='Taz_num', how='left')
                .merge(forecast_point_urban, on='Taz_num', how='left')
                .merge(forecast_point_SCHOOLDISTRICT, on='Taz_num', how='left')
                .merge(forecast_point_PUMA, on='Taz_num', how='left')
                .merge(forecast_point_jerusalem_city, on='Taz_num', how='left'))

# Rename columns# Rename columns with flipped names
forecast.rename(columns={'ENG_NAME_nafa': 'zonetype'}, inplace=True)
forecast.rename(columns={'puma2040_csv_DISTRICT': 'DISTRICT'}, inplace=True)
forecast.rename(columns={'BaseProjections2040_csv_urban': 'urban'}, inplace=True)
forecast.rename(columns={'puma2040_csv_PUMA': 'PUMA'}, inplace=True)
forecast.rename(columns={'puma2040_csv_SCHOOLDISTRICT': 'SCHOOLDISTRICT'}, inplace=True)


# Data processing for feature columns
forecast.loc[forecast['main_sector'] == 'Palestinian', 'zonetype'] = 'Palestinian'
forecast['in_jerusalem_metropolin'] = 1
forecast.loc[forecast['jeru_metro'] == 0, 'in_jerusalem_metropolin'] = 0
forecast['yosh'] = 0
forecast.loc[forecast['zonetype'] == 'Judea and Samaria', 'yosh'] = 1

# Add constant columns
forecast['REGION'] = 1
forecast['slop'] = 0

forecast = forecast.set_index('Taz_num')
forecast['Taz_num'] = forecast.index

In [9]:
#### שינוי עמודה Muni_Heb
forecast=change_Muni_Heb_to_Muni_Eng(software_data_folder_location, forecast)

In [10]:
#### ייצוא שכבת אזורי תנועה לבקרת לקוח
save_shp_path=r'{}\For_approval\{}_taz_for_approval.shp'.format(client_data_folder_location,file_date)
    
col=['Taz_num','Name_hebre','Muni_Eng', 'main_sector', 'classification_name','Student_by_Classrooms','geometry']

forecast[col].to_file(save_shp_path,index=False,encoding='UTF-8')

C:\Users\dpere\AppData\Local\Temp\ipykernel_29880\938773556.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  forecast[col].to_file(save_shp_path,index=False,encoding='UTF-8')


In [ ]:
#### מצב קיים לבקרה
forecast_2020=export_status_exists(create_forecast_basic_location, forecast_version_basic_folder_location, forecast, software_data_folder_location,client_data_folder_location, file_date)

#### העלאת מרכיבי טבלת אינדקס
index=uploading_index_table(forecast, client_data_folder_location, index_file_name)

## חלוקה לאזורי תנועה של התכניות
divided_index=division_into_traffic_zones(index,forecast)

### שכבת אינדקס
index_layer=index_layer_fun(divided_index, software_data_folder_location)

#### ייצוא שכבת אינדקס לבקרת לקוח
index_layer_for_client_control=export_index_layer(index_layer,client_data_folder_location,file_date,forecast_version)

### חישוב תחזית
#### הוספת תוספת בעקבות האינדקס
forecast=adding_an_addition(index_layer,forecast,forecast_2020,software_data_folder_location,client_data_folder_location,forecast_version)

#ייצוא תוצאות
export_forecast(forecast, client_data_folder_location, file_date, forecast_version,software_data_folder_location)

print('Done')